# Contents of the notebook

To be able to use the packages ``bw2regional`` and ``bw2_lcimpact``, I modified their code. I did not ask for a pulling request on Chris Mutel's Github. Therefore, this notebook is temporary and contains the code that you should include in the packages of your environment to make this tutorial work. You can copy past it instead of the code you downloaded from Chris Mutel repository.

The objectives of this notebook are:
1. To modify the code of bw2data. The adress is:
   ``C:\Users\``yourname``\AppData\Local\anaconda3\envs\brightway_regional_2\Lib\site-packages\bw2data``
2. To modify the code of bw2regional. The adress is:
   ``C:\Users\``yourname``\AppData\Local\anaconda3\envs\brightway_regional_2\Lib\site-packages\bw2regional``
3. To modify the code of bw2_lcimpact. The adress is:
   ``C:\Users\``yourname``\AppData\Local\anaconda3\envs\brightway_regional_2\Lib\site-packages\bw2_lcimpact``

# Modification of the bw2data code

## utils.py

For the ``get_geocollection(location, default_global_location=False):`` function, I removed attaching the geocollection ``World`` to the ``glo`` location. In fact, it leads to "global cfs" being regionalised although they are not. Further, it leads brightway regional to calculate un-necessary intersections.

In [ ]:
import collections
import itertools
import numbers
import os
import random
import re
import string
import urllib
import warnings
import zipfile
from io import StringIO
from pathlib import Path

import stats_arrays as sa

from .configuration import labels
from .errors import MultipleResults, NotFound, UnknownObject, ValidityError
from .fatomic import open

DOWNLOAD_URL = "https://brightway.dev/data/"


def safe_filename(*args, **kwargs):
    raise DeprecationWarning("`safe_filename` has been moved to `bw_processing`")


def maybe_path(x):
    return Path(x) if x else x


def natural_sort(l):
    """Sort the given list in the way that humans expect, e.g. 9 before 10."""
    # http://nedbatchelder.com/blog/200712/human_sorting.html#comments
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split("([0-9]+)", key)]
    return sorted(l, key=alphanum_key)


def random_string(length=8):
    """Generate a random string of letters and numbers.

    Args:
        * *length* (int): Length of string, default is 8

    Returns:
        A string (not unicode)

    """
    return "".join(
        random.choice(string.ascii_letters + string.digits) for i in range(length)
    )


def combine_methods(name, *ms):
    """Combine LCIA methods by adding duplicate characterization factors.

    Args:
        * *ms* (one or more method id tuples): Any number of method ids, e.g.
        ``("my method", "wow"), ("another method", "wheee")``.

    Returns:
        The new Method instance.

    """
    from . import Method, methods

    data = {}
    units = set([methods[tuple(x)]["unit"] for x in ms])
    for m in ms:
        for key, cf, geo in Method(m).load():
            data[(key, geo)] = data.get((key, geo), 0) + cf
    meta = {
        "description": "Combination of the following methods: "
        + ", ".join([str(x) for x in ms]),
        "unit": list(units)[0] if len(units) == 1 else "Unknown",
    }
    data = [(key, cf, geo) for (key, geo), cf in data.items()]
    method = Method(name)
    method.register(**meta)
    method.write(data)
    return method


def clean_exchanges(data):
    """Make sure all exchange inputs are tuples, not lists."""

    def tupleize(value):
        for exc in value.get("exchanges", []):
            exc["input"] = tuple(exc["input"])
        return value

    return {key: tupleize(value) for key, value in data.items()}


POSITIVE_DISTRIBUTIONS = {2, 6, 8, 9, 10}


def as_uncertainty_dict(value):
    """Given either a number or a ``stats_arrays`` uncertainty dict, return an uncertainty dict"""
    if isinstance(value, dict):
        if (
            value.get("amount", 0) < 0
            and (
                value.get("uncertainty_type") in POSITIVE_DISTRIBUTIONS
                or value.get("uncertainty type") in POSITIVE_DISTRIBUTIONS
            )
            and "negative" not in value
        ):
            value["negative"] = True
        return value
    try:
        return {"amount": float(value)}
    except:
        raise TypeError(
            "Value must be either an uncertainty dict. or number"
            " (got %s: %s)" % (type(value), value)
        )


def uncertainify(data, distribution=None, bounds_factor=0.1, sd_factor=0.1):
    """
    Add some rough uncertainty to exchanges.

    .. warning:: This function only changes exchanges with no uncertainty type or uncertainty type ``UndefinedUncertainty``, and does not change production exchanges!

    Can only apply normal or uniform uncertainty distributions; default is uniform. Distribution, if specified, must be a ``stats_array`` uncertainty object.

    ``data`` is a LCI data dictionary.

    If using the normal distribution:

    * ``sd_factor`` will be multiplied by the mean to calculate the standard deviation.
    * If no bounds are desired, set ``bounds_factor`` to ``None``.
    * Otherwise, the bounds will be ``[(1 - bounds_factor) * mean, (1 + bounds_factor) * mean]``.

    If using the uniform distribution, then the bounds are ``[(1 - bounds_factor) * mean, (1 + bounds_factor) * mean]``.

    Returns the modified data.
    """
    assert distribution in {
        None,
        sa.UniformUncertainty,
        sa.NormalUncertainty,
    }, "``uncertainify`` only supports normal and uniform distributions"
    assert (
        bounds_factor is None or bounds_factor * 1.0 > 0
    ), "bounds_factor must be a positive number"
    assert sd_factor * 1.0 > 0, "sd_factor must be a positive number"

    for key, value in data.items():
        for exchange in value.get("exchanges", []):
            if (exchange.get("type") in labels.technosphere_positive_edge_types) or (
                exchange.get("uncertainty type", sa.UndefinedUncertainty.id)
                != sa.UndefinedUncertainty.id
            ):
                continue
            if exchange["amount"] == 0:
                continue

            if bounds_factor is not None:
                exchange.update(
                    {
                        "minimum": (1 - bounds_factor) * exchange["amount"],
                        "maximum": (1 + bounds_factor) * exchange["amount"],
                    }
                )
                if exchange["amount"] < 0:
                    exchange["minimum"], exchange["maximum"] = (
                        exchange["maximum"],
                        exchange["minimum"],
                    )

            if distribution == sa.NormalUncertainty:
                exchange.update(
                    {
                        "uncertainty type": sa.NormalUncertainty.id,
                        "loc": exchange["amount"],
                        "scale": abs(sd_factor * exchange["amount"]),
                    }
                )
            else:
                assert (
                    bounds_factor is not None
                ), "must specify bounds_factor for uniform distribution"
                exchange.update(
                    {
                        "uncertainty type": sa.UniformUncertainty.id,
                    }
                )
    return data


def recursive_str_to_unicode(data, encoding="utf8"):
    """Convert the strings inside a (possibly nested) python data structure to unicode strings using `encoding`."""
    # Adapted from
    # http://stackoverflow.com/questions/1254454/fastest-way-to-convert-a-dicts-keys-values-from-unicode-to-str
    if isinstance(data, str):
        return data
    elif isinstance(data, bytes):
        return str(data, encoding)  # Faster than str.encode
    elif isinstance(data, collections.abc.Mapping):
        return dict(
            map(recursive_str_to_unicode, data.items(), itertools.repeat(encoding))
        )
    elif isinstance(data, collections.abc.Iterable):
        return type(data)(
            map(recursive_str_to_unicode, data, itertools.repeat(encoding))
        )
    else:
        return data


def combine_databases(name, *dbs):
    """Combine databases into new database called ``name``."""
    pass


def merge_databases(parent_db, other):
    """Merge ``other`` into ``parent_db``, including updating exchanges.

    All databases must be SQLite databases.

    ``parent_db`` and ``other`` should be the names of databases.

    Doesn't return anything."""
    from . import databases
    from .backends import (
        ActivityDataset,
        ExchangeDataset,
        SQLiteBackend,
        sqlite3_lci_db,
    )
    from .database import Database

    assert parent_db in databases
    assert other in databases

    first = Database(parent_db)
    second = Database(other)

    if type(first) != SQLiteBackend or type(second) != SQLiteBackend:
        raise ValidityError("Both databases must be `SQLiteBackend`")

    first_codes = {
        obj.code
        for obj in ActivityDataset.select().where(ActivityDataset.database == parent_db)
    }
    second_codes = {
        obj.code
        for obj in ActivityDataset.select().where(ActivityDataset.database == other)
    }
    if first_codes.intersection(second_codes):
        raise ValidityError("Duplicate codes - can't merge databases")

    with sqlite3_lci_db.atomic():
        ActivityDataset.update(database=parent_db).where(
            ActivityDataset.database == other
        ).execute()
        ExchangeDataset.update(input_database=parent_db).where(
            ExchangeDataset.input_database == other
        ).execute()
        ExchangeDataset.update(output_database=parent_db).where(
            ExchangeDataset.output_database == other
        ).execute()

    Database(parent_db).process()
    del databases[other]


def download_file(filename, directory="downloads", url=None):
    """Download a file and write it to disk in ``downloads`` directory.

    If ``url`` is None, uses the Brightway2 data base URL. ``url`` should everything up to the filename, such that ``url`` + ``filename`` is the valid complete URL to download from.

    Streams download to reduce memory usage.

    Args:
        * *filename* (str): The filename to download.
        * *directory* (str, optional): Directory to save the file. Created if it doesn't already exist.
        * *url* (str, optional): URL where the file is located, if not the default Brightway data URL.

    Returns:
        The path of the created file.

    """
    from . import projects

    assert isinstance(directory, str), "`directory` must be a string"
    dirpath = projects.request_directory(directory)
    filepath = dirpath / filename
    download_path = (url if url is not None else DOWNLOAD_URL) + filename
    with urllib.request.urlopen(download_path) as response, open(
        filepath, "wb"
    ) as out_file:
        if response.status != 200:
            raise NotFound(
                "URL {} returns status code {}.".format(download_path, response.status)
            )
        chunk = 128 * 1024
        while True:
            segment = response.read(chunk)
            if not segment:
                break
            out_file.write(segment)
    return filepath


def set_data_dir(dirpath, permanent=True):
    """Set the Brightway2 data directory to ``dirpath``.

    If ``permanent`` is ``True``, then set ``dirpath`` as the default data directory.

    Creates ``dirpath`` if needed. Also creates basic directories, and resets metadata.

    """
    warnings.warn(
        "`set_data_dir` is deprecated; use `projects.set_current('my "
        "project name')` for a new project space.",
        DeprecationWarning,
    )


def switch_data_directory(dirpath):
    from .projects import ProjectDataset, SubstitutableDatabase

    if dirpath == bw.projects._base_data_dir:
        print("dirpath already loaded")
        return
    try:
        assert os.path.isdir(dirpath)
        bw.projects._base_data_dir = dirpath
        bw.projects._base_logs_dir = os.path.join(dirpath, "logs")
        # create folder if it does not yet exist
        if not os.path.isdir(bw.projects._base_logs_dir):
            os.mkdir(bw.projects._base_logs_dir)
        # load new brightway directory
        bw.projects.db = SubstitutableDatabase(
            os.path.join(bw.projects._base_data_dir, "projects.db"), [ProjectDataset]
        )
        print("Loaded brightway2 data directory: {}".format(bw.projects._base_data_dir))

    except AssertionError:
        print('Could not access directory specified "dirpath"')


def create_in_memory_zipfile_from_directory(path):
    # Based on http://stackoverflow.com/questions/2463770/python-in-memory-zip-library
    memory_obj = StringIO()
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    zf = zipfile.ZipFile(memory_obj, "a", zipfile.ZIP_DEFLATED, False)
    for filename in files:
        zf.writestr(filename, open(os.path.join(path, filename)).read())
    # Mark the files as having been created on Windows so that
    # Unix permissions are not inferred as 0000
    for zfile in zf.filelist:
        zfile.create_system = 0
    zf.close()
    memory_obj.seek(0)
    return memory_obj


def get_node(**kwargs):
    from . import databases
    from .backends import ActivityDataset as AD
    from .subclass_mapping import NODE_PROCESS_CLASS_MAPPING

    def node_class(database_name):
        return NODE_PROCESS_CLASS_MAPPING[
            databases[database_name].get("backend", "sqlite")
        ]

    mapping = {
        "id": AD.id,
        "code": AD.code,
        "database": AD.database,
        "location": AD.location,
        "name": AD.name,
        "product": AD.product,
        "type": AD.type,
    }

    qs = AD.select()
    for key, value in kwargs.items():
        try:
            qs = qs.where(mapping[key] == value)
        except KeyError:
            continue

    candidates = [node_class(obj.database)(obj) for obj in qs]

    extended_search = any(key not in mapping for key in kwargs)
    if extended_search:
        if "database" not in kwargs:
            warnings.warn(
                "Given search criteria very broad; try to specify at least a database"
            )
        candidates = [
            obj
            for obj in candidates
            if all(
                obj.get(key) == value
                for key, value in kwargs.items()
                if key not in mapping
            )
        ]
    if len(candidates) > 1:
        raise MultipleResults(
            "Found {} results for the given search".format(len(candidates))
        )
    elif not candidates:
        raise UnknownObject
    return candidates[0]


def get_activity(key=None, **kwargs):
    """Support multiple ways to get exactly one activity node.

    ``key`` can be an integer or a key tuple."""
    from .backends import Activity

    # Includes subclasses
    if isinstance(key, Activity):
        return key
    elif isinstance(key, tuple):
        kwargs["database"] = key[0]
        kwargs["code"] = key[1]
    elif isinstance(key, numbers.Integral):
        kwargs["id"] = key
    return get_node(**kwargs)


def get_geocollection(location, default_global_location=False):
    """conservative approach to finding geocollections. Won't guess about ecoinvent or other databases."""
    if not location:
        if default_global_location:
            return "world"
        else:
            return None
    elif isinstance(location, tuple):
        return location[0]
    #elif isinstance(location, str) and (
    #    len(location) == 2 or location.lower() == "glo"
    #):
    #    return "world"
    else:
        return None

## method.py

I used the ``false`` condition instead of the ``true`` one in ``get_geocollection(third(elem), default_global_location=False)``. In fact, the methods are not necessarily regionalized. What's more, sometimes, they are assigned by default the ``world`` location, eventhough they should be linked to another geocollection. It leads to brightway regional calculating intersections when it is not necessary.

In [ ]:
from . import config, geomapping, methods
from .backends.schema import get_id
from .errors import UnknownObject
from .ia_data_store import ImpactAssessmentDataStore
from .utils import as_uncertainty_dict, get_geocollection
from .validate import ia_validator


class Method(ImpactAssessmentDataStore):
    """A manager for an impact assessment method. This class can register or deregister methods, write intermediate data, process data to parameter arrays, validate, and copy methods.

    The Method class never holds intermediate data, but it can load or write intermediate data. The only attribute is *name*, which is the name of the method being managed.

    Instantiation does not load any data. If this method is not yet registered in the metadata store, a warning is written to ``stdout``.

    Methods are hierarchally structured, and this structure is preserved in the method name. It is a tuple of strings, like ``('ecological scarcity 2006', 'total', 'natural resources')``.

    The data schema for IA methods is:

    .. code-block:: python

            Schema([Any(
                [valid_tuple, maybe_uncertainty],         # site-generic
                [valid_tuple, maybe_uncertainty, object]  # regionalized
            )])

    where:
        * *valid_tuple* (tuple): A dataset identifier, like ``("biosphere", "CO2")``.
        * *maybe_uncertainty* (uncertainty dict or number): Either a number or an uncertainty dictionary.
        * *object* (object, optional) is a location identifier, used only for regionalized LCIA.

    Args:
        * *name* (tuple): Name of impact assessment method to manage.

    """

    _metadata = methods
    validator = ia_validator
    matrix = "characterization_matrix"

    def add_geomappings(self, data):
        geomapping.add({x[2] for x in data if len(x) == 3})

    def process_row(self, row):
        """Given ``(flow, amount, maybe location)``, return a dictionary for array insertion."""
        try:
            return {
                **as_uncertainty_dict(row[1]),
                "row": get_id(row[0]),
                "col": (
                    geomapping[row[2]]
                    if len(row) >= 3
                    else geomapping[config.global_location]
                ),
            }
        except UnknownObject:
            raise UnknownObject(
                "Can't find flow `{}`, specified in CF row `{}` for method `{}`".format(
                    {tuple(row[0]) if isinstance(row[0], list) else row[0]}, row, self.name
                )
            )
        except KeyError:
            if len(row) >= 3 and row[2] not in geomapping:
                raise UnknownObject(
                    "Can't find location `{}`, specified in CF row `{}` for method `{}`".format(
                        {row[2]}, row, self.name
                    )
                )
            elif config.global_location not in geomapping:
                raise UnknownObject(
                    "Can't find default global location! It's supposed to be `{}`, but this isn't in the `geomapping`".format(
                        config.global_location
                    )
                )

    def write(self, data, process=True):
        """Serialize intermediate data to disk.

        Sets the metadata key ``num_cfs`` automatically."""
        if self.name not in self._metadata:
            self.register()
        self.metadata["num_cfs"] = len(data)

        third = lambda x: x[2] if len(x) == 3 else None

        geocollections = {
            get_geocollection(third(elem), default_global_location=False) ## NR : I changed this because else, it will attach a "world" geocollection to the LCIA eventhough the methods are not necessarily regionalized.
            for elem in data
        }
        if None in geocollections:
            geocollections.discard(None)

        self.metadata["geocollections"] = sorted(geocollections)
        self._metadata.flush()
        super(Method, self).write(data, process=process)

    def process(self, **extra_metadata):
        try:
            extra_metadata["global_index"] = geomapping[config.global_location]
        except KeyError:
            raise KeyError(
                "Can't find default global location! It's supposed to be `{}`, defined in `config`, but this isn't in the `geomapping`".format(
                    config.global_location
                )
            )
        super().process(**extra_metadata)

# Modification of the ``bw2regional`` code

## __init__.py

I added the importation of the new ``GlobalLocLCA``, allowing to compute the impacts of elementary flows of activities in the [GLO] location.

In [ ]:
__all__ = (
    "cg",
    "calculate_needed_intersections",
    "create_ecoinvent_collections",
    "create_empty_intersection",
    "create_restofworlds_collections",
    "create_world_collections",
    "divide_by_area",
    "extension_tables",
    "ExtensionTable",
    "ExtensionTablesLCA",
    "geocollections",
    "get_spatial_dataset_kind",
    "GlobalLocLCA", # NR : I added this line to compute the impacts of elementary flows of activities in the [GLO] location.
    "hash_collection",
    "import_from_pandarus",
    "import_regionalized_cfs",
    "Intersection",
    "calculate_intersection",
    "intersections",
    "label_activity_geocollections",
    "Loading",
    "loadings",
    "OneSpatialScaleLCA",
    "PandarusRemote",
    "remote",
    "reset_all_geo",
    "reset_geo_meta",
    "sha256",
    "topocollections",
    "Topography",
    "TwoSpatialScalesLCA",
    "TwoSpatialScalesWithGenericLoadingLCA",
    "raster_as_extension_table",
)

# ignore future warning from pandas that we can't fix
import warnings

from .version import version as __version__

warnings.simplefilter(action="ignore", category=FutureWarning)

from constructive_geometries import ConstructiveGeometries

cg = ConstructiveGeometries()

from bw2data import config

from .topography import Topography
from .loading import Loading
from .meta import (
    extension_tables,
    geocollections,
    intersections,
    loadings,
    topocollections,
)
from .intersection import Intersection, calculate_needed_intersections
from .xtables import ExtensionTable
from .databases import label_activity_geocollections
from .density import divide_by_area
from .lca import (
    ExtensionTablesLCA,
    GlobalLocLCA, # NR : I added this line to compute the impacts of elementary flows of activities in the [GLO] location.
    OneSpatialScaleLCA,
    TwoSpatialScalesLCA,
    TwoSpatialScalesWithGenericLoadingLCA,
)

from .base_data import (
    create_ecoinvent_collections,
    create_restofworlds_collections,
    create_world_collections,
)
from .gis_tasks import calculate_intersection, raster_as_extension_table
from .hashing import sha256
from .pandarus import import_from_pandarus
from .pandarus_remote import PandarusRemote, remote
from .utils import (
    create_empty_intersection,
    get_spatial_dataset_kind,
    hash_collection,
    import_regionalized_cfs,
    reset_all_geo,
    reset_geo_meta,
)

config.metadata.extend(
    [extension_tables, geocollections, topocollections, intersections, loadings]
)

## base_data.py


The major changes I made were:
1. To remove the links for downloading the "world" and "ecoinvent" geocollections from an online repository and instead to write the adress of a local file
2. To add a geocollection related to the RoW because it was empty.
3. To add a tupple ("world", k) in the ``Topography`` of the "world" geocollection. The explanations are in the section "3.3 databases.py"
4. I changed the field names of the geocollections to avoid issues when calculating intersections between spatial vectors using geopandas.

Add the adress of the different files of the LCI maps for each comments "NR:" of the script 

In [ ]:
import bz2
import json
import os
import warnings

import rower
from bw2data.utils import download_file

from . import Topography, cg, geocollections, topocollections
from .hashing import sha256

COUNTRIES = {
    "AD",
    "AE",
    "AF",
    "AG",
    "AI",
    "AL",
    "AM",
    "AO",
    "AQ",
    "AR",
    "AS",
    "AT",
    "AU",
    "AW",
    "AX",
    "AZ",
    "BA",
    "BB",
    "BD",
    "BE",
    "BF",
    "BG",
    "BH",
    "BI",
    "BJ",
    "BL",
    "BM",
    "BN",
    "BO",
    "BR",
    "BS",
    "BT",
    "BW",
    "BY",
    "BZ",
    "CA",
    "CD",
    "CF",
    "CG",
    "CH",
    "CI",
    "CK",
    "CL",
    "CM",
    "CN",
    "CO",
    "CR",
    "CU",
    "CV",
    "CW",
    "CY",
    "CZ",
    "DE",
    "DJ",
    "DK",
    "DM",
    "DO",
    "DZ",
    "EC",
    "EE",
    "EG",
    "EH",
    "ER",
    "ES",
    "ET",
    "FI",
    "FJ",
    "FK",
    "FM",
    "FO",
    "FR",
    "GA",
    "GB",
    "GD",
    "GE",
    "GG",
    "GH",
    "GI",
    "GL",
    "GM",
    "GN",
    "GQ",
    "GR",
    "GS",
    "GT",
    "GU",
    "GW",
    "GY",
    "HK",
    "HM",
    "HN",
    "HR",
    "HT",
    "HU",
    "ID",
    "IE",
    "IL",
    "IM",
    "IN",
    "IO",
    "IQ",
    "IR",
    "IS",
    "IT",
    "JE",
    "JM",
    "JO",
    "JP",
    "KE",
    "KG",
    "KH",
    "KI",
    "KM",
    "KN",
    "KP",
    "KR",
    "KW",
    "KY",
    "KZ",
    "LA",
    "LB",
    "LC",
    "LI",
    "LK",
    "LR",
    "LS",
    "LT",
    "LU",
    "LV",
    "LY",
    "MA",
    "MC",
    "MD",
    "ME",
    "MF",
    "MG",
    "MH",
    "MK",
    "ML",
    "MM",
    "MN",
    "MO",
    "MP",
    "MR",
    "MS",
    "MT",
    "MU",
    "MV",
    "MW",
    "MX",
    "MY",
    "MZ",
    "NA",
    "NC",
    "NE",
    "NF",
    "NG",
    "NI",
    "NL",
    "NO",
    "NP",
    "NR",
    "NU",
    "NZ",
    "OM",
    "PA",
    "PE",
    "PF",
    "PG",
    "PH",
    "PK",
    "PL",
    "PM",
    "PN",
    "PR",
    "PS",
    "PT",
    "PW",
    "PY",
    "QA",
    "RO",
    "RS",
    "RU",
    "RW",
    "SA",
    "SB",
    "SC",
    "SD",
    "SE",
    "SG",
    "SH",
    "SI",
    "SK",
    "SL",
    "SM",
    "SN",
    "SO",
    "SR",
    "SS",
    "ST",
    "SV",
    "SX",
    "SY",
    "SZ",
    "TC",
    "TD",
    "TF",
    "TG",
    "TH",
    "TJ",
    "TL",
    "TM",
    "TN",
    "TO",
    "TR",
    "TT",
    "TV",
    "TW",
    "TZ",
    "UA",
    "UG",
    "UM",
    "US",
    "UY",
    "UZ",
    "VA",
    "VC",
    "VE",
    "VG",
    "VI",
    "VN",
    "VU",
    "WF",
    "WS",
    "XK",
    "YE",
    "ZA",
    "ZM",
    "ZW",
}


def create_world_collections():
    geocollections["world"] = {
        "filepath": os.path.join(""), #NR: you have to enter the adress of the world map (countries.gpkg)
        "field": "isotwolettercode_world",
    }
    topocollections["world"] = {
        "geocollection": "world",
        "filepath": str(cg.faces_fp),
        "field": "id",
    }
    topo_data = {("world",k): v for k, v in cg.data.items() if k in COUNTRIES} #NR: I added the tupple ("world", k) instead of only k
    Topography("world").write(topo_data)


def create_ecoinvent_collections():
    geocollections["ecoinvent"] = {
        "filepath": os.path.join(""),#NR: you have to enter the adress of the ecoinvent map (all-ecoinvent.gpkg)
        "field": "shortname_ecoinvent",
    }
    topocollections["ecoinvent"] = {
        "geocollection": "ecoinvent",
        "filepath": str(cg.faces_fp),
        "field": "id",
    }
    topo_data = {
        ("ecoinvent", k): v
        for k, v in cg.data.items()
        if k != "__all__" and "RoW" not in k and k not in COUNTRIES
    }
    Topography("ecoinvent").write(topo_data)


def create_restofworlds_collections():
    filepath = os.path.join(
        rower.DATAPATH, "ecoinvent generic", "rows-topomapping.json.bz2"
    )
    with bz2.BZ2File(filepath) as f:
        rower_data = json.load(f)

    if sha256(cg.faces_fp) != rower_data["metadata"]["sha256"]:
        warnings.warn(
            "Inconsistent `rower` and `constructive_geometries` packages. Skipping 'RoW' creation"
        )
        return

    print("Creating `rower` 'RoW' geo/topocollections")
    geocollections["RoW"] = {
        "filepath" : os.path.join(""), #NR: you have to enter the adress of the RoW map (row.gpkg)
        "field" : "name_RoW",
        "kind" : "vector"
        }
    topocollections["RoW"] = {
        "geocollection": "RoW",
        "filepath": cg.faces_fp,
        "field": "id",
    }
    topo_data = {("RoW", k): v for k, v in rower_data["data"]}
    Topography("RoW").write(topo_data)


## databases.py

The major change I did was labeling the location of the activities happening in a country with the tupple ("world","iso two letters country code") instead of the string "iso two letters country code". 
``calculate_intersections(engine=geopanda)`` calculates the intersections between two polygons. The results are written as 
``("geocollection 1", "id2") ("geocollection2","id2") "area of the intersection (m2)"``.
As such, if we don't use tupples ("geocollection 1", "id2") for renaming the locations of the activities of the databases, it leads to trouble populating the different matrices of regionalized LCA..

In [ ]:
import warnings

import pyprind
from bw2data import Database, databases

from . import Topography, geocollections, topocollections
from .base_data import COUNTRIES


def label_activity_geocollections(name):
    """Add geocollections to activity ``location`` fields.

    ``name`` is the name of an existing LCI database."""
    assert name in databases, "{} not found".format(name)
    assert "world" in geocollections, "Please run `create_world_collections` first"

    #NR: I added a world set in order to be able after to label databases locations with the tuple ("world", "two letter code of country")
    world = (
        {x[1] for x in Topography("world").load()}
        if "world" in topocollections
        else set()
    )
    
    ecoinvent = (
        {x[1] for x in Topography("ecoinvent").load()}
        if "ecoinvent" in topocollections
        else set()
    )
    RoWs = (
        {x[1] for x in Topography("RoW").load()} if "RoW" in topocollections else set()
    )

    db = Database(name)
    searchable = db.metadata.get("searchable")
    if searchable:
        db.make_unsearchable()

    found_geocollections = set()

    locations = {x["location"] for x in db}
    assert "RoW" not in locations, "`RoW` found; use `rower` to label Rest-of-Worlds"

    for act in pyprind.prog_bar(db):
        if isinstance(act["location"], tuple):
            found_geocollections.add(act["location"][0])
        elif act["location"] in COUNTRIES: 
            act["location"] = ("world", act["location"]) #NR: labeling databases locations with the tuple ("world", "iso two letter code of country")
            act.save()
            found_geocollections.add("world")
        elif act["location"] == "GLO":
            found_geocollections.add("world") #NR: I did not label it with a tupple ("world","GLO") as the other world location because the GLO location is special (maybe we could modify it)
        elif act["location"] in RoWs:
            act["location"] = ("RoW", act["location"])
            act.save()
            found_geocollections.add("RoW")
        elif act["location"] in ecoinvent:
            act["location"] = ("ecoinvent", act["location"])
            act.save()
            found_geocollections.add("ecoinvent")
        else:
            warnings.warn(
                (
                    "Location {} in {} not understood; please add geocollection"
                    " manually, and add to databases[name]['geocollections']"
                ).format(act["location"], act.key)
            )

    if searchable:
        db.make_searchable()
    db.process()

    db.metadata["regionalized"] = True
    db.metadata["geocollections"] = sorted(found_geocollections)
    databases.flush()

## lca\\__init__.py

I added a new class ``GlobalLocLCA`` that allow the evaluation of the impacts of elementary flows of activities happening in the [GLO] location

In [ ]:
# -*- coding: utf-8 -*-
from .extension_tables import ExtensionTablesLCA
from .global_loc import GlobalLocLCA
from .one_spatial_scale import OneSpatialScaleLCA
from .two_spatial_scales import TwoSpatialScalesLCA
from .two_spatial_scales_weighting import TwoSpatialScalesWithGenericLoadingLCA

## lca\\base_class.py

I added a new module in the ``RegionalizationBase`` class : ``create_glob_characterization_matrix``, which will allow the computation of elementary flows of activities in [GLO] location. This new module is the same as ``create_regionalized_characterization_matrix``.

In [ ]:
import itertools
from functools import partial

import matrix_utils as mu
import numpy as np
from bw2calc.lca import LCA
from bw2data import Database, Method, databases, get_activity, methods
from scipy.sparse import coo_matrix, csr_matrix

from ..errors import MissingIntersection, SiteGenericMethod, UnprocessedDatabase
from ..export import create_geodataframe
from ..intersection import Intersection
from ..meta import intersections
from ..utils import dp


def get_dependent_databases(demand_dict):
    """Demand can be activitiy ids or tuple keys."""
    db_labels = [
        x[0] if isinstance(x, tuple) else get_activity(x)["database"]
        for x in demand_dict
    ]
    return set.union(*[Database(label).find_graph_dependents() for label in db_labels])


def annotate_flow(flow_id, _):
    flow = get_activity(id=flow_id)
    return {
        "flow_name": flow.get("name", ""),
        "flow_unit": flow.get("unit", ""),
        "flow_categories": str(flow.get("categories", "")),
    }


class RegionalizationBase(LCA):
    def __init__(self, demand, *args, **kwargs):
        self.databases = get_dependent_databases(demand)
        self.extra_data_objs = kwargs.pop("extra_data_objs", [])
        super(RegionalizationBase, self).__init__(demand, *args, **kwargs)

    def get_inventory_geocollections(self):
        """Get the set of all needed inventory geocollections.

        Raise UnprocessedDatabase if any database is missing the required metadata."""
        missing, present = [], set()
        for database in self.databases:
            if "geocollections" not in databases[database]:
                missing.append(database)
            else:
                present.update(set(databases[database]["geocollections"]))
        if missing:
            raise UnprocessedDatabase(
                "Database(s) {} don't specify their geocollections.".format(missing)
            )
        return present

    def get_ia_geocollections(self):
        """Retrieve the geocollections linked to the impact assessment method"""
        ia_gc = set(methods[self.method]["geocollections"])
        if not ia_gc:
            raise SiteGenericMethod
        return ia_gc

    def create_inventory_mapping_matrix(self):
        """Get inventory mapping matrix, **M**, which maps inventory activities to inventory locations. Rows are inventory activities and columns are inventory spatial units.

        Uses ``self.technosphere_mm.row_mapper`` and ``self.databases``.

        Creates ``self.inv_mapping_mm``, ``self.inv_mapping_matrix``, and ``self.dicts.inv_spatial``/

        """
        self.inv_mapping_mm = mu.MappedMatrix(
            packages=[
                dp(Database(x).filepath_processed()) for x in self.databases
            ] + self.extra_data_objs,
            matrix="inv_geomapping_matrix",
            use_arrays=self.use_arrays,
            use_distributions=self.use_distributions,
            seed_override=self.seed_override,
            row_mapper=self.technosphere_mm.col_mapper,
        )
        self.inv_mapping_matrix = self.inv_mapping_mm.matrix
        self.dicts.inv_spatial = partial(self.inv_mapping_mm.col_mapper.to_dict)

    def needed_intersections(self):
        """Figure out which ``Intersection`` objects are needed bsed on ``self.inventory_geocollections`` and ``self.ia_geocollections``.

        Raise ``MissingIntersection`` if an intersection is required, but not available."""
        required = list(
            itertools.product(self.inventory_geocollections, self.ia_geocollections)
            )
        for obj in required:
            if obj not in intersections:
                raise MissingIntersection(
                    "Intersection {} needed but not found".format(obj)
                )
        return required

    def create_geo_transform_matrix(self):
        """Get geographic transform matrix **G**, which gives the intersecting areas of inventory and impact assessment spatial units. Rows are inventory spatial units, and columns are impact assessment spatial units.

        Uses ``self.inv_spatial_dict`` and ``self.ia_spatial_dict``.

        Returns:
            * ``geo_transform_params``: Parameter array with row/col of inventory and IA locations
            * ``geo_transform_matrix``: The matrix **G**

        """
        self.geo_transform_mm = mu.MappedMatrix(
            packages=[
                dp(Intersection(name).filepath_processed())
                for name in self.needed_intersections()
            ] + self.extra_data_objs,
            matrix="intersection_matrix",
            use_arrays=self.use_arrays,
            use_distributions=self.use_distributions,
            seed_override=self.seed_override,
            col_mapper=self.reg_cf_mm.row_mapper,
            row_mapper=self.inv_mapping_mm.col_mapper,
        )
        self.geo_transform_matrix = self.geo_transform_mm.matrix

    def create_regionalized_characterization_matrix(self, row_mapper=None):
        """Get regionalized characterization matrix, **R**, which gives location- and biosphere flow-specific characterization factors.

        Rows are impact assessment spatial units, and columns are biosphere flows. However, we build it transverse and transpose it, as the characterization matrix indices are provided that way.

        Uses ``self._biosphere_dict`` and ``self.method``.

        Returns:
            * ``reg_cf_params``: Parameter array with row/col of IA locations/biosphere flows
            * ``ia_spatial_dict``: Dictionary linking impact assessment locations to matrix rows
            * ``reg_cf_matrix``: The matrix **R**

        """
        self.reg_cf_mm = mu.MappedMatrix(
            packages=[
                dp(Method(self.method).filepath_processed())
            ] + self.extra_data_objs,
            matrix="characterization_matrix",
            use_arrays=self.use_arrays,
            use_distributions=self.use_distributions,
            seed_override=self.seed_override,
            col_mapper=self.biosphere_mm.row_mapper,
            row_mapper=row_mapper,
            transpose=True,
        )
        self.reg_cf_matrix = self.reg_cf_mm.matrix
        if row_mapper is None:
            self.dicts.ia_spatial = partial(self.reg_cf_mm.row_mapper.to_dict)

    def create_glob_characterization_matrix(self, row_mapper=None): # NR : I added this module in order to characterize the elementary flows happening at the [GLO] location
            """ Creating a characterization matrix for elementary flows of the [GLO] location"""
            self.glob_cf_mm = mu.MappedMatrix(
                packages=[
                    dp(Method(self.method).filepath_processed())
                ] + self.extra_data_objs,
                matrix="characterization_matrix",
                use_arrays=self.use_arrays,
                use_distributions=self.use_distributions,
                seed_override=self.seed_override,
                col_mapper=self.biosphere_mm.row_mapper,
                row_mapper=row_mapper,
                transpose=True,
            )
            self.glob_cf_matrix = self.glob_cf_mm.matrix
            if row_mapper is None:
                self.dicts.ia_spatial = partial(self.glob_cf_mm.row_mapper.to_dict)

    def create_loading_matrix(self):
        """Get diagonal regionalized loading matrix, **L**, which gives location-specific background loading factors. Dimensions are impact assessment spatial units.

        Uses ``self.dicts.ia_spatial``.

        """
        self.loading_mm = mu.MappedMatrix(
            packages=[
                dp(self.loading.filepath_processed())
            ] + self.extra_data_objs,
            matrix="loading_matrix",
            use_arrays=self.use_arrays,
            use_distributions=self.use_distributions,
            seed_override=self.seed_override,
            diagonal=True,
            row_mapper=self.reg_cf_mm.row_mapper,
        )
        self.loading_matrix = self.loading_mm.matrix

    def _results_new_scale(self, matrix, flow):
        # self.fix_spatial_dictionaries()
        if flow is not None:
            try:
                row_index = self.biosphere_dict[flow]
                matrix = matrix[row_index, :]
            except KeyError:
                raise ValueError("Flow {} not in biosphere dict".format(flow))
        else:
            # using matrix.sum() converts to dense numpy matrix
            nc = matrix.shape[0]
            summer = csr_matrix(
                (np.ones(nc), np.arange(nc), np.array((0, nc), dtype=int))
            )
            matrix = summer * matrix
        return matrix

    def results_ia_spatial_scale(self):
        raise NotImplementedError("Must be defined in subclasses")

    def results_inv_spatial_scale(self):
        raise NotImplementedError("Must be defined in subclasses")

    def __geodataframe(
        self, matrix, sum_flows, annotate_flows, col_dict, used_geocollections, cutoff
    ):
        if sum_flows:
            matrix = coo_matrix(matrix.sum(axis=0))
            annotate_flows = None
        elif annotate_flows:
            annotate_flows = annotate_flow

        return create_geodataframe(
            matrix=matrix,
            used_geocollections=used_geocollections,
            row_dict=self.dicts.biosphere,
            col_dict=col_dict,
            attribute_adder=annotate_flows,
            cutoff=cutoff,
        )

    def geodataframe_xtable_spatial_scale(
        self, sum_flows=True, annotate_flows=None, cutoff=None
    ):
        if not hasattr(self, "results_xtable_spatial_scale"):
            raise NotImplementedError

        matrix = self.results_xtable_spatial_scale()
        return self.__geodataframe(
            matrix=matrix,
            sum_flows=sum_flows,
            annotate_flows=annotate_flows,
            col_dict=self.dicts.xtable_spatial,
            used_geocollections=self.xtable_geocollections,
            cutoff=cutoff,
        )

    def geodataframe_ia_spatial_scale(
        self, sum_flows=True, annotate_flows=None, cutoff=None
    ):
        matrix = self.results_ia_spatial_scale()
        return self.__geodataframe(
            matrix=matrix,
            sum_flows=sum_flows,
            annotate_flows=annotate_flows,
            col_dict=self.dicts.ia_spatial,
            used_geocollections=self.ia_geocollections,
            cutoff=cutoff,
        )

    def geodataframe_inv_spatial_scale(
        self, sum_flows=True, annotate_flows=None, cutoff=None
    ):
        matrix = self.results_inv_spatial_scale()
        return self.__geodataframe(
            matrix=matrix,
            sum_flows=sum_flows,
            annotate_flows=annotate_flows,
            col_dict=self.dicts.inv_spatial,
            used_geocollections=self.inventory_geocollections,
            cutoff=cutoff,
        )

## lca\\global_loc.py

The ``GlobalLocLCA`` class will account for elementary flows of activities in the [GLO] location. It uses the RegionalizationBase class. It is a copy of the OneSpatialScaleLCA class with some modifications, e.g. removal of the error impeding the computation of the LCA with shared spatial scale when there are two spatial scales and modification of LCIA_calculation with the use of ``self.glob_cf_matrix``. The ``glob_cf_matrix`` is only filled with CFs related to the [GLO] location.

In [ ]:
from bw2data import methods

from .base_class import RegionalizationBase

class GlobalLocLCA(RegionalizationBase):
    matrix_labels = [
        "biosphere_mm",
        "inv_mapping_mm",
        "glob_cf_mm",
        "technosphere_mm",
    ]
    
    def __init__(self, *args, **kwargs):
        """Perform LCA calculation, for the elementary flows at the global scale, which are not computed in the TwoSpatialScalesLCA
        """
        super(GlobalLocLCA, self).__init__(*args, **kwargs)
        if self.method not in methods:
            raise ValueError("Must pass valid `method` name")
        self.inventory_geocollections = self.get_inventory_geocollections()
        self.ia_geocollections = self.get_ia_geocollections()

    def load_lcia_data(self):
        self.create_inventory_mapping_matrix()
        self.create_glob_characterization_matrix(self.inv_mapping_mm.col_mapper)
  
    def lcia_calculation(self):
        """Do LCA calculation for elementary flows at the global scale.

        Creates ``self.characterized_inventory``.

        """
        self.characterized_inventory = (
            self.inv_mapping_matrix * self.glob_cf_matrix
        ).T.multiply(self.inventory)

    def results_ia_spatial_scale(self):
        raise NotImplementedError("No separate IA spatial scale")

    def results_inv_spatial_scale(self):
        if not hasattr(self, "characterized_inventory"):
            raise ValueError("Must do lcia calculation first")
        return self.reg_cf_matrix.T.multiply(self.inventory * self.inv_mapping_matrix)

# Modification of the ``bw2_lcimpact``

## water.py

I renamed the field name "id" to "id_water_eq_sw" for the geocollections related to the impacts of water consumption on ecosystem quality to avoid conflicts of names.

In [ ]:
import os

from .base import LCIA, data_dir, fiona, geocollections, regionalized

"""When is water not water? When it is in water!

We are assessing freshwater consumption, and so some water flows are excluded."""

SURFACE_WATER = [
    ("Fresh water (obsolete)", ("water", "surface water"), -1),
    ("Water", ("water",), -1),
    ("Water", ("water", "surface water"), -1),
    ("Water, cooling, unspecified natural origin", ("natural resource", "in water"), 1),
    ("Water, lake", ("natural resource", "in water"), 1),
    ("Water, river", ("natural resource", "in water"), 1),
    (
        "Water, turbine use, unspecified natural origin",
        ("natural resource", "in water"),
        1,
    ),
    ("Water, unspecified natural origin", ("natural resource", "in water"), 1),
]

GROUND_WATER = [
    ("Water", ("water", "ground-"), -1),
    # ('Water', ('water', 'ground-, long-term')),
    ("Water, unspecified natural origin", ("natural resource", "in ground"), 1),
    ("Water, well, in ground", ("natural resource", "in water"), 1),
]


class Water(LCIA):
    def _water_flows(self, kind="all"):
        mapping = {
            "all": SURFACE_WATER + GROUND_WATER,
            "surface": SURFACE_WATER,
            "ground": GROUND_WATER,
        }
        flows = mapping[kind]

        for act in self.db:
            name, categories = act["name"], tuple(act["categories"])
            for x, y, sign in flows:
                if name == x and categories == y:
                    yield act.key, sign


class WaterHumanHealthMarginal(Water):
    vector_ds = os.path.join(data_dir, "water_hh.gpkg")
    geocollection = "watersheds-hh"
    column = "HH"

    name = ("LC-IMPACT", "Water Use", "Human Health", "Marginal")
    global_cf = 1.8e-7
    unit = "DALY/m3"
    description = """The impact assessment method for assessing water consumption concerning the area of protection of human health is described based on Pfister et al. (2009) for the impact pathway (marginal CF), Pfister and Hellweg (2011) for uncertainty assessment, and Pfister and Bayer (2013) for average CFs.

    Only the 'certain' level of uncertainty is provided."""
    url = "http://lc-impact.eu/human-health-water-stress"

    @regionalized
    def setup_geocollections(self):
        if self.geocollection not in geocollections:
            geocollections[self.geocollection] = {
                "filepath": self.vector_ds,
                "field": "BAS34S_ID",
            }

    def global_cfs(self):
        for key, sign in self._water_flows():
            yield ((key, self.global_cf * sign, "GLO"))

    @regionalized
    def regional_cfs(self):
        water_flows = list(self._water_flows())

        for obj in self.global_cfs():
            yield obj

        with fiona.Env():
            with fiona.open(self.vector_ds) as src:
                for feat in src:
                    for key, sign in water_flows:
                        yield (
                            key,
                            feat["properties"][self.column]
                            * 1e-9
                            * sign,  # Convert km3 to m3
                            (self.geocollection, feat["properties"]["BAS34S_ID"]),
                        )


class WaterHumanHealthAverage(WaterHumanHealthMarginal):
    name = ("LC-IMPACT", "Water Use", "Human Health", "Average")
    global_cf = 1.3e-7
    column = "HH_AVG"


class WaterEcosystemQualityCertain(Water):
    vector_ds = os.path.join(data_dir, "water_eq_sw_core.gpkg")
    geocollection = "watersheds-eq-sw-certain"
    column = "val"

    name = (
        "LC-IMPACT",
        "Water Use",
        "Ecosystem Quality",
        "Surface Water",
        "Marginal",
        "Certain",
    )
    global_cf = 1.63e-13
    unit = "PDF·yr/m3"
    description = """The description of the impact assessment approach for quantifying impacts from water consumption on biodiversity is based on Verones et al. (submitted), which is a continuation from Verones et al. (2013a) and Verones et al. (2013b), as well as Chaudhary et al. (2015)."""
    url = "http://lc-impact.eu/ecosystem-quality-water-stress"

    _flows_label = "surface"

    @regionalized
    def setup_geocollections(self):
        if self.geocollection not in geocollections:
            geocollections[self.geocollection] = {
                "filepath": self.vector_ds,
                "field": "id_water_eq_sw", #NR: I added "_water_eq_sw" to the field name
            }

    def global_cfs(self):
        for key, sign in self._water_flows(self._flows_label):
            yield ((key, self.global_cf * sign, "GLO"))

    @regionalized
    def regional_cfs(self):
        water_flows = list(self._water_flows(self._flows_label))

        for obj in self.global_cfs():
            yield obj

        with fiona.Env():
            with fiona.open(self.vector_ds) as src:
                for feat in src:
                    for key, sign in water_flows:
                        yield (
                            key,
                            feat["properties"][self.column] * sign,
                            (self.geocollection, feat["properties"]["id_water_eq_sw"]), #NR: I added "_water_eq_sw" to the field name
                        )


class WaterEcosystemQualityAll(WaterEcosystemQualityCertain):
    vector_ds = os.path.join(data_dir, "water_eq_sw_extended.gpkg")
    geocollection = "watersheds-eq-sw-all"

    name = (
        "LC-IMPACT",
        "Water Use",
        "Ecosystem Quality",
        "Surface Water",
        "Marginal",
        "All",
    )
    global_cf = 1.65e-13

    _flows_label = "all"
